In [1]:
import urllib
url = (
    "https://raw.githubusercontent.com/openforcefield/protein-ligand-benchmark/"
    "8c94c0dcc892dfd77992567294b1ff31c62e8695/plbenchmark/sample_data/2020-08-26_mcl1_sample/"
)

urllib.request.urlretrieve(url + "01_protein/crd/protein.pdb", "protein.pdb")
urllib.request.urlretrieve(url + "02_ligands/lig_23/crd/lig_23.sdf", "lig_23.sdf")

('lig_23.sdf', <http.client.HTTPMessage at 0x108bce3d0>)

In [5]:
import openmm.app
import openmm
import openmm.unit

print('Loading...')
pdb = openmm.app.PDBFile('protein.pdb')
forcefield = openmm.app.ForceField('amber99sb.xml', 'tip3p.xml')
modeller = openmm.app.Modeller(pdb.topology, pdb.positions)
print('Adding hydrogens...')
modeller.addHydrogens(forcefield)
print('Adding solvent...')
modeller.addSolvent(forcefield, model='tip3p', padding=1*openmm.unit.nanometer)
#print('Minimizing...')
system = forcefield.createSystem(
        modeller.topology, 
        nonbondedMethod=openmm.app.PME,
        nonbondedCutoff=9.0 * openmm.unit.angstrom,
        switchDistance=8.0 * openmm.unit.angstrom,
        constraints=openmm.app.HBonds
    )

Loading...
Adding hydrogens...
Adding solvent...


In [6]:
from openff.toolkit import Molecule, ForceField
from openff.interchange import Interchange
ligand = Molecule.from_file("lig_23.sdf")
sage = ForceField("openff-2.2.1.offxml")
ligand_interchange = sage.create_interchange(ligand.to_topology())

In [7]:
protein_interchange = Interchange.from_openmm(
    system=system, 
    topology=modeller.topology,
    positions=modeller.getPositions(),
    box_vectors=modeller.topology.getPeriodicBoxVectors(),
    )
# Ensure the non-bonded settings match the `Interchange` created with Sage
protein_interchange['vdW'].cutoff = ligand_interchange['vdW'].cutoff
protein_interchange['vdW'].switch_width = ligand_interchange['vdW'].switch_width
protein_interchange['Electrostatics'].cutoff = ligand_interchange['Electrostatics'].cutoff

In [8]:
complex_interchange = ligand_interchange.combine(protein_interchange)

/Users/jeffreywagner/micromamba/envs/demo2/lib/python3.11/site-packages/openff/interchange/operations/_combine.py:77: InterchangeCombinationWarning: Interchange object combination is complex and likely to produce strange results outside of a limited set of use cases it has been tested in. Any workflow using this method is not guaranteed to be suitable for production or stable between versions. Use with extreme caution and thoroughly validate results!
  warnings.warn(
/Users/jeffreywagner/micromamba/envs/demo2/lib/python3.11/site-packages/openff/interchange/operations/_combine.py:57: InterchangeCombinationWarning: Found electrostatics 1-4 scaling factors of 5/6 with slightly different rounding (0.833333 and 0.8333333333). This likely stems from OpenFF using more digits in rounding 1/1.2. The value of 0.8333333333 will be used, which may or may not introduce small errors. 
  warnings.warn(


In [10]:
integrator = openmm.LangevinIntegrator(
    300 * openmm.unit.kelvin,
    1 / openmm.unit.picosecond,
    0.002 * openmm.unit.picoseconds,
)

# Under the hood, this creates *OpenMM* `System` and `Topology` objects, then combines them together
simulation = complex_interchange.to_openmm_simulation(integrator=integrator)

In [13]:
dcd_reporter = openmm.app.DCDReporter(file="trajectory.dcd", reportInterval=100)
simulation.reporters.append(dcd_reporter)

In [14]:
# This will take a minute since we carelessly 
# added water without the ligand present
simulation.minimizeEnergy()

In [15]:
simulation.context.setVelocitiesToTemperature(300 * openmm.unit.kelvin)
simulation.runForClockTime(.5 * openmm.unit.minute)

In [16]:
import mdtraj
import nglview
trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(complex_interchange.to_openmm_topology())
)

view = nglview.show_mdtraj(trajectory)#.image_molecules())
view.add_representation("line", selection="protein or water")

view

NGLWidget(max_frame=17)